In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Choose the dataset based on the roll number
roll_no = 207  # Replace with your roll number
mod_value = roll_no % 3
if mod_value == 0:
    dataset_name = "STL10"
    full_trainset = datasets.STL10(root='./data', split='train', download=True, transform=transform_train)
    full_testset = datasets.STL10(root='./data', split='test', download=True, transform=transform_test)
elif mod_value == 1:
    dataset_name = "SVHN"
    full_trainset = datasets.SVHN(root='./data', split='train', download=True, transform=transform_train)
    full_testset = datasets.SVHN(root='./data', split='test', download=True, transform=transform_test)
else:
    dataset_name = "FashionMNIST"
    full_trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform_train)
    full_testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform_test)

# Define the data loaders with sub-sampling
train_indices = torch.randperm(len(full_trainset))[:10000]
test_indices = torch.randperm(len(full_testset))[:2000]

trainset = Subset(full_trainset, train_indices)
testset = Subset(full_testset, test_indices)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet101 model
model = torchvision.models.resnet101(pretrained=True)

# Freeze convolutional layers
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for our datasets
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizers
optimizers = {
    "Adam": optim.Adam(model.parameters(), lr=0.001),
    "Adagrad": optim.Adagrad(model.parameters(), lr=0.01),
    "Adadelta": optim.Adadelta(model.parameters(), lr=1.0),
    "RMSprop": optim.RMSprop(model.parameters(), lr=0.001)
}

# Training function
def train(optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    return train_loss, train_acc

# Testing function
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    test_loss /= len(testloader)
    test_acc = 100. * correct / total
    return test_loss, test_acc

# Training loop for each optimizer
epochs = 10
for optimizer_name, optimizer in optimizers.items():
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    print(f"Training with {optimizer_name}")
    for epoch in range(epochs):
        train_loss, train_acc = train(optimizer)
        test_loss, test_acc = test()

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)

        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

    # Plotting
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs + 1), test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss Curve ({optimizer_name})')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy')
    plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title(f'Accuracy Curve ({optimizer_name})')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Report final top-5 test accuracy
model.eval()
top5_correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.topk(5, 1, True, True)
        targets = targets.view(-1, 1)
        top5_correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

top5_acc = 100. * top5_correct / total
print(f"Final Top-5 Test Accuracy: {top5_acc:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Training with Adam
